In [1]:
import numpy as np
import pandas as pd

# Basic setup
zones = ["Core", "ULB", "110 Villages"]
base_pop = {"Core": 6480000, "ULB": 4810000, "110 Villages": 2820000}
lpcd = {"Core": 150, "ULB": 135, "110 Villages": 120}
growth_rate = 0.02
rainfall_means = [2, 3, 10, 20, 50, 100, 150, 120, 80, 30, 10, 5]
temp_means = [22, 24, 28, 30, 32, 29, 28, 27, 26, 25, 23, 22]
supply_limits = {"Core": 900, "ULB": 600, "110 Villages": 300}
scada_coverage = {"Core": "Full", "ULB": "Partial", "110 Villages": "None"}
holidays = ["01-26", "08-15", "10-02", "12-25"]  # Fixed-date holidays

dates = pd.date_range("2021-01-01", "2023-12-31", freq="D")
np.random.seed(42)
records = []

for zone in zones:
    for date in dates:
        year_diff = date.year - 2021
        month = date.month
        pop = base_pop[zone] * ((1 + growth_rate) ** year_diff)
        temp = np.random.normal(temp_means[month - 1], 2.5)
        rain = max(0, np.random.normal(rainfall_means[month - 1], 20))

        base_demand = (pop * lpcd[zone]) / 1_000_000  # MLD
        noise = base_demand * (0.05 * np.random.randn())
        temp_effect = (temp - 28) * 0.5
        rain_effect = -0.05 * rain
        demand = base_demand + noise + temp_effect + rain_effect

        # 🔹 Weekend / Holiday Adjustment
        is_weekend = date.weekday() >= 5
        is_holiday = date.strftime("%m-%d") in holidays
        if is_weekend or is_holiday:
            demand *= np.random.uniform(0.9, 0.95)  # 5–10% reduction

        # 🔹 Unaccounted Water Loss (Theft/Leak)
        unaccounted_loss = demand * np.random.uniform(0.05, 0.15)
        demand += unaccounted_loss

        # 🔹 Pipe Burst (Rare spike event, ~0.5% chance)
        pipe_burst = np.random.rand() < 0.005
        if pipe_burst:
            demand += np.random.uniform(30, 60)  # Sudden burst: +30–60 MLD

        demand = round(max(0, demand), 2)

        slum_ratio = 0.12 if zone == "Core" else 0.05
        slum_demand = round((pop * slum_ratio * 30) / 1_000_000, 2)
        groundwater = round(demand * 0.2, 2)
        shortage = demand > base_demand * 1.05
        deficit = max(0, demand - base_demand)
        tankers = int(deficit * 1000 / 10) if shortage else 0

        records.append([
            date, zone, int(pop), round(temp, 2), round(rain, 2), base_demand,
            demand, slum_demand, groundwater, unaccounted_loss, pipe_burst,
            shortage, tankers, scada_coverage[zone]
        ])

# Final dataset
columns = [
    "Date", "Zone", "Population", "Temperature_C", "Rainfall_mm", "Base_Demand_MLD",
    "Water_Demand_MLD", "Slum_Consumption_MLD", "Groundwater_Use_MLD",
    "Unaccounted_Loss_MLD", "Pipe_Burst_Event", "Shortage", "Tankers_Dispatched", "SCADA_Coverage"
]

df = pd.DataFrame(records, columns=columns)
df.to_csv("Enhanced_Urban_Water_Demand_Dataset.csv", index=False)
print("✅ Enhanced dataset saved as 'Enhanced_Urban_Water_Demand_Dataset.csv'")


✅ Enhanced dataset saved as 'Enhanced_Urban_Water_Demand_Dataset.csv'


In [2]:
# 📦 Install dependencies
!pip install gradio pandas matplotlib scikit-learn --quiet

# ✅ Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# ✅ Load Dataset
df = pd.read_csv("Enhanced_Urban_Water_Demand_Dataset.csv")
df["Zone_Code"] = df["Zone"].map({"Core": 0, "ULB": 1, "110 Villages": 2})

# ✅ Model Training
features = ["Population", "Temperature_C", "Rainfall_mm", "Zone_Code"]
X = df[features]
y = df["Water_Demand_MLD"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)


# ✅ Fixed Supply per Zone (MLD) and Tanker Capacity
supply_limits = {"Core": 900, "ULB": 600, "110 Villages": 300}
tanker_capacity_kl = 10

# ✅ Gradio Function: Predict + Allocate
def predict_allocation(zone, population, temperature, rainfall):
    zone_code = {"Core": 0, "ULB": 1, "110 Villages": 2}[zone]
    input_data = [[population, temperature, rainfall, zone_code]]

    # Predict Demand
    predicted_demand = model.predict(input_data)[0]
    supply = supply_limits[zone]
    deficit = max(0, predicted_demand - supply)
    tankers = int(deficit * 1000 / tanker_capacity_kl)

    # Summary Output
    summary = (
        f"📍 Zone: {zone}\n"
        f"👥 Population: {int(population)}\n"
        f"🌡️ Temperature: {temperature} °C\n"
        f"🌧️ Rainfall: {rainfall} mm\n\n"
        f"💧 Predicted Water Demand: {predicted_demand:.2f} MLD\n"
        f"🚰 Supply Available: {supply} MLD\n"
        f"⚠️ Deficit: {deficit:.2f} MLD\n"
        f"🚛 Tankers Required: {tankers} (10 KL each)\n"
    )

    # Visualization
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.bar(["Demand", "Supply"], [predicted_demand, supply], color=["blue", "green"])
    ax.set_ylabel("MLD")
    ax.set_title(f"{zone} - Demand vs Supply")
    plt.tight_layout()

    return summary, fig

# ✅ Gradio UI
gr.Interface(
    fn=predict_allocation,
    inputs=[
        gr.Dropdown(["Core", "ULB", "110 Villages"], label="Select Zone"),
        gr.Number(label="Population", value=5000000),
        gr.Number(label="Temperature (°C)", value=30),
        gr.Number(label="Rainfall (mm)", value=20)
    ],
    outputs=[
        gr.Textbox(label="Tanker Allocation Summary"),
        gr.Plot(label="Demand vs Supply Chart")
    ],
    title="💧 Urban Water Demand + 🚛 Tanker Allocator",
    description="Select a zone and enter current population, temperature, and rainfall to predict daily water demand and tanker requirements."
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd74e1c3f5b4f014f9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
# 📦 Install requirements
!pip install gradio pandas matplotlib scikit-learn --quiet

# ✅ Imports
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# ✅ Load dataset
df = pd.read_csv("Enhanced_Urban_Water_Demand_Dataset.csv")
df["Zone_Code"] = df["Zone"].map({"Core": 0, "ULB": 1, "110 Villages": 2})

# ✅ Train model
X = df[["Population", "Temperature_C", "Rainfall_mm", "Zone_Code"]]
y = df["Water_Demand_MLD"]
model = RandomForestRegressor(n_estimators=200, random_state=42).fit(X, y)

# ✅ Fixed supply
supply_limits = {"Core": 900, "ULB": 600, "110 Villages": 300}
tanker_capacity_kl = 10

# ✅ Predict + Allocate with leakage & tanker cap
def predict_with_constraints(zone, population, temp, rain, leakage_pct, max_tankers):
    zone_code = {"Core": 0, "ULB": 1, "110 Villages": 2}[zone]
    base_demand = model.predict([[population, temp, rain, zone_code]])[0]

    leakage_mld = base_demand * (leakage_pct / 100)
    adjusted_demand = base_demand + leakage_mld

    supply = supply_limits[zone]
    deficit = max(0, adjusted_demand - supply)
    raw_tankers = int(deficit * 1000 / tanker_capacity_kl)
    capped_tankers = min(raw_tankers, max_tankers)

    summary = (
        f"📍 Zone: {zone}\n"
        f"👥 Population: {int(population)}\n"
        f"🌡️ Temperature: {temp} °C\n"
        f"🌧️ Rainfall: {rain} mm\n"
        f"💦 Leakage Assumed: {leakage_pct}% → +{leakage_mld:.2f} MLD\n\n"
        f"💧 Adjusted Demand: {adjusted_demand:.2f} MLD\n"
        f"🚰 Supply: {supply} MLD\n"
        f"⚠️ Deficit: {deficit:.2f} MLD\n"
        f"🚛 Raw Tankers: {raw_tankers}\n"
        f"🔒 Tankers Dispatched (Capped): {capped_tankers}"
    )

    # Plot
    fig, ax = plt.subplots(figsize=(2, 2))
    ax.bar(["Demand", "Supply"], [adjusted_demand, supply], color=["blue", "green"])
    ax.set_ylabel("MLD")
    ax.set_title(f"{zone} - Adjusted Demand vs Supply")
    plt.tight_layout()
    return summary, fig

# ✅ Launch Gradio app
gr.Interface(
    fn=predict_with_constraints,
    inputs=[
        gr.Dropdown(["Core", "ULB", "110 Villages"], label="Select Zone"),
        gr.Number(label="Population", value=5000000),
        gr.Number(label="Temperature (°C)", value=30),
        gr.Number(label="Rainfall (mm)", value=20),
        gr.Slider(minimum=0, maximum=20, step=1, label="Leakage (%)", value=5),
        gr.Number(label="Max Tankers per Day (cap)", value=5000)
    ],
    outputs=[
        gr.Textbox(label="Prediction & Tanker Allocation Summary"),
        gr.Plot(label="Demand vs Supply Chart")
    ],
    title="💧 Urban Water Demand + 🚛 Tanker Allocator (Leakage & Cap)",
    description="Predict adjusted water demand including leakage and limit daily tanker dispatch."
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://acb38a960a5c8ca65b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Split test data (if not already done)
from sklearn.model_selection import train_test_split

X = df[["Population", "Temperature_C", "Rainfall_mm", "Zone_Code"]]
y = df["Water_Demand_MLD"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Re-train model (if needed)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# 🔍 Evaluation Metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 📊 Print Results
print("🔍 Model Evaluation Metrics:")
print(f"✅ R² Score        : {r2:.4f}  (1.0 is perfect)")
print(f"📉 Mean Absolute Error (MAE) : {mae:.2f} MLD")
print(f"📉 Root Mean Squared Error (RMSE): {rmse:.2f} MLD")


🔍 Model Evaluation Metrics:
✅ R² Score        : 0.9604  (1.0 is perfect)
📉 Mean Absolute Error (MAE) : 42.68 MLD
📉 Root Mean Squared Error (RMSE): 57.80 MLD
